In [ ]:
## One-Dim Incremental learning

In [1]:
from typing import ForwardRef
import torch
import torch.nn as nn
import torchvision as tv
import torch.nn.functional as F
from torch.autograd import Variable

class OneDimNet(nn.Module):
    def __init__(self):
        super(OneDimNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=10, kernel_size=3, stride=2) # length = 479
        self.max_pool1=nn.MaxPool1d(kernel_size=3, stride=2) # length = 239
        self.conv2=nn.Conv1d(10, 20, 3, 2) # length = 119
        self.max_pool2 = nn.MaxPool1d(3, 2) # length = 59
        self.conv3=nn.Conv1d(20, 40, 3, 2) # length = 29

        self.liner1 = nn.Linear(40 * 14, 120)
        self.liner2 = nn.Linear(120, 84)
        self.liner3 = nn.Linear(84, 4)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)
        x = F.relu(self.conv3(x))
        

        x = x.view(-1, 40 * 14)
        x = F.relu(self.liner1(x))
        x = F.relu(self.liner2(x))
        x = self.liner3(x)

        return x

tensor([[ 0.0971, -0.0596,  0.0462,  0.0827]], grad_fn=<AddmmBackward0>)

In [31]:
output = net.forward(input)

In [26]:
conv1=nn.Conv1d(20, 40, 3, 2) # length = 29
input = torch.randn(1, 29, 20) # batch, length, channels

In [27]:
input = input.permute(0,2,1)
input = Variable(input)

In [28]:
out = conv1(input)

In [32]:
out.size()

torch.Size([1, 40, 14])

In [18]:
sin = np.array(np.load('sin.npy'))
tri = np.array(np.load('tri.npy'))
squ = np.array(np.load('squ.npy'))
x = np.array([sin,tri,squ])
x.shape
y = np.array([[0],[1],[2]])
y
y.shape


(3, 1)

In [33]:
import torch
import torch.nn as nn
import numpy as np

# 效果不太好
sin = np.array(np.load('sin.npy'))
tri = np.array(np.load('tri.npy'))
squ = np.array(np.load('squ.npy'))

# 构建输入集
# x = np.mat('0 0;'
#            '0 1;'
#            '1 0;'
#            '1 1')
x = np.array([sin,tri,squ])
x = torch.tensor(x).float()
y = np.array([[0],[1],[2]])
y = torch.tensor(y).float()

# 搭建网络
myNet = nn.Sequential(
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
    nn.Sigmoid()
)
print(myNet)

# 设置优化器
optimzer = torch.optim.SGD(myNet.parameters(), lr=0.05)
loss_func = nn.MSELoss()

for epoch in range(5000):
    out = myNet(x)
    loss = loss_func(out, y)  # 计算误差
    optimzer.zero_grad()  # 清除梯度
    loss.backward()
    optimzer.step()


print(myNet(x).data)


Sequential(
  (0): Linear(in_features=256, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=10, bias=True)
  (3): ReLU()
  (4): Linear(in_features=10, out_features=1, bias=True)
  (5): Sigmoid()
)
tensor([[0.0070],
        [0.9969],
        [1.0000]])


In [34]:
input=np.load('sin_30.npy')
x = torch.tensor(input).float()
(myNet(x).data)


tensor([0.0173])

In [48]:
# from typing import ForwardRef
import torch
import torch.nn as nn
import torchvision as tv
import torch.nn.functional as F
from torch.autograd import Variable

class OneDimNet(nn.Module):
    def __init__(self):
        super(OneDimNet, self).__init__()
        ## input = 1 x 227 x 1 (Row, Column, channel)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0) # weight of size [96, 1, 11] (out chan, in chan, ker_size)
        self.max_pool1=nn.MaxPool1d(kernel_size=3, stride=2) # length = 
        self.conv2=nn.Conv1d(96, 256, 5, 1, 2) # length = 119
        self.max_pool2 = nn.MaxPool1d(3, 2) # length = 59
        self.conv3=nn.Conv1d(256, 384, 3, 1, 1) # length = 29
        self.conv4=nn.Conv1d(384, 384, 3, 1, 1)
        self.conv5=nn.Conv1d(384, 256, 3, 1, 1)


        self.fc1 = nn.Linear(256, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)
        x = F.relu(self.conv3(x))
        

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.liner1(x))
        x = F.relu(self.liner2(x))
        x = self.liner3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features

    # optimzer = torch.optim.SGD(OneDimNet.parameters(), lr=0.05)
    # loss_func = nn.MSELoss()

    # for epoch in range(5000):
    #     out = OneDimNet(x)
    #     loss = loss_func(out, y)  # 计算误差
    #     optimzer.zero_grad()  # 清除梯度
    #     loss.backward()
    #     optimzer.step()

In [49]:
net=OneDimNet()
print(net)

input = Variable(torch.randn(3, 1, 227)) # batch, channel, Height, Width

optimzer = torch.optim.SGD(OneDimNet.parameters(), lr=0.05)
loss_func = nn.MSELoss()

for epoch in range(5000):
    out = OneDimNet(x)
    loss = loss_func(out, y)  # 计算误差
    optimzer.zero_grad()  # 清除梯度
    loss.backward()
    optimzer.step()

OneDimNet(
  (conv1): Conv1d(1, 96, kernel_size=(11,), stride=(4,))
  (max_pool1): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(96, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (max_pool2): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(256, 384, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv4): Conv1d(384, 384, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv5): Conv1d(384, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=256, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=3, bias=True)
)


In [54]:
conv1 = nn.Conv1d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0) 
input = Variable(torch.randn(1, 1, 227)) # batch, channel, Height, Width
out = conv1(input)
print(out)

tensor([[[ 0.0509, -0.8549,  0.0343,  ...,  0.4890,  0.0732, -0.3196],
         [ 0.8909,  0.1815, -0.5570,  ...,  0.0067,  0.2550,  0.3093],
         [-1.2353,  0.0278,  0.5363,  ..., -0.8812, -0.7001,  0.7730],
         ...,
         [-0.0428,  0.3096,  0.0481,  ...,  0.8138,  1.2837, -0.3760],
         [ 1.5505,  0.4601, -0.5719,  ...,  0.7353,  0.5914, -0.4212],
         [-0.7276, -0.0724,  0.5273,  ..., -0.2496, -0.8922, -0.6197]]],
       grad_fn=<SqueezeBackward1>)
